## This script adds combat abilities, re-loads all the csv's, and puts them in the SQLite db

In [37]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
abilities = pd.read_csv('processed_data/ability.csv')
combat = pd.read_csv('processed_data/combat.csv')
pokemon_abilities = pd.read_csv('processed_data/pokemon_ability.csv')
pokemon = pd.read_csv('processed_data/pokemon.csv')
trainers_pokemon = pd.read_csv('processed_data/trainer_pokemon.csv')
trainers = pd.read_csv('processed_data/trainer.csv')
types = pd.read_csv('processed_data/type.csv')

In [39]:
#add combat abilities
ability_pairs = {}
for index, row in pokemon_abilities.iterrows():
    ability_pairs[row['pokemon_id']] = int(row['ability_id'])

winning_ability = []
for index, row in combat.iterrows():
    try:
        winning_ability.append(ability_pairs[row['winning_pokemon_id']])
    except:
        winning_ability.append(None)

combat['winning_ability'] = winning_ability
combat['winning_ability'] = combat['winning_ability'].astype('int')

combat

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [36]:
combat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   combat_id           1995 non-null   int64  
 1   winning_pokemon_id  1995 non-null   int64  
 2   winning_trainer_id  1995 non-null   int64  
 3   losing_pokemon_id   1995 non-null   int64  
 4   losing_trainer_id   1995 non-null   int64  
 5   winning_ability     1984 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 93.6 KB


In [30]:
#save to a single excel file
writer = pd.ExcelWriter('processed_data/full_dataset.xlsx', engine='xlsxwriter')

pokemon.to_excel(writer, sheet_name='pokemon')
types.to_excel(writer, sheet_name='type')
abilities.to_excel(writer, sheet_name='ability')
trainers.to_excel(writer, sheet_name='trainer')
combat.to_excel(writer, sheet_name='combat')
pokemon_abilities.to_excel(writer, sheet_name='pokemon_ability')
trainers_pokemon.to_excel(writer, sheet_name='trained_pokemon')

writer.save()

In [31]:
# to sqlite
def connect_sqlite():
    con = sqlite3.connect('../data.db')
    return con, con.cursor()

#send to SQLite db
con, cur = connect_sqlite()

pokemon.to_sql("pokemon", con, if_exists='replace')
types.to_sql("type", con, if_exists='replace')
abilities.to_sql("ability", con, if_exists='replace')
trainers.to_sql("trainer", con, if_exists='replace')
combat.to_sql("combat", con, if_exists='replace')
pokemon_abilities.to_sql("pokemon_ability", con, if_exists='replace')
trainers_pokemon.to_sql("trained_pokemon", con, if_exists='replace')